<a href="https://colab.research.google.com/github/MikeSalnikov/ML_for_business/blob/main/Step3_cource_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Итоговый проект (урок № 9)

## Step 3: FLASK

In [2]:
#!pip install flask==2.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 KB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 4.3 MB/s eta 0:00:00
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1
  Attempting uninstall: itsdangerous
    Found existing installation: itsdangerous 1.1.0
    Uninstalling itsdangerous-1.1.0:
      Successfully uninstalled itsdangerous-1.1.0
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 1.0.1
    Un

In [3]:
# from flask_ngrok import run_with_ngrok # для google colab

from flask import Flask, request, jsonify
import pandas as pd
import dill

### Создаю сервис для обработки запросов к модели

In [5]:
# загружаю обученную модель: 

with open('logreg_pipeline.dill', 'rb') as in_strm:
    model = dill.load(in_strm)

In [7]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

#### Запускаю сервис и не глушу его, пока работаю 

In [ ]:
# обработчики и запуск Flask: 

app = Flask(__name__)
# run_with_ngrok(app)  # Start ngrok when app is run # для google colab


@app.route("/", methods=["GET"])
def general():
    return "Welcome to prediction process"

@app.route('/predict', methods=['POST'])
def predict():
    data = {"success": False}

    # ensure an image was properly uploaded to our endpoint
    convex_area, eccentricity, perimeter = "", "", ""
    request_json = request.get_json()

    if request_json["convex_area"]:
        convex_area = request_json['convex_area']

    if request_json["eccentricity"]:
        eccentricity = request_json['eccentricity']
        
    if request_json["perimeter"]:
        perimeter = request_json['perimeter']
    
    print(convex_area, eccentricity, perimeter)  
    preds = model.predict_proba(pd.DataFrame({"convex_area": [convex_area],
                                              "eccentricity": [eccentricity],
                                              "perimeter": [perimeter]}))
    data["predictions"] = preds[:, 1][0]
    # indicate that the request was a success
    data["success"] = True
    print('OK')

    # return the data dictionary as a JSON response
    return jsonify(data)


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


### Тестовый клиент

In [14]:
# пример данных:

convex_area, eccentricity, perimeter = (16343, 0.90705806, 518.8930054)

body = {
        'convex_area': convex_area,
        'eccentricity': eccentricity,
        'perimeter': perimeter
        }

In [ ]:
with app.test_client() as t:
    response = t.post('/predict', json=body)
    json_data = response.get_json()

json_data